In [28]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [29]:
df = pd.read_csv("../data/cleaned_fake_jobs.csv")


In [30]:
print(df.shape)
df.head()

(17880, 2)


,cleaned_desc,fraudulent
0,foodafastgrowingjamesbeardawardwinningonlinefo...,0
1,organisedfocusedvibrantawesomedoyouhaveapassio...,0
2,ourclientlocatedinhoustonisactivelyseekinganex...,0
3,thecompanyesrienvironmentalsystemsresearchinst...,0
4,jobtitleitemizationreviewmanagerlocationfortwo...,0


In [31]:
df = df.dropna(subset=['cleaned_desc'])

In [32]:
df = df[df['cleaned_desc'].str.strip() != '']

In [33]:
df = df.reset_index(drop=True)

In [34]:
texts = df['cleaned_desc'].tolist()
labels = df['fraudulent'].tolist()

In [35]:
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(texts).toarray()

In [36]:
X_padded = pad_sequences(X, maxlen=250, padding='post', truncating='post')

In [ ]:
y = np.array(labels)

In [38]:
assert X_padded.shape[0] == y.shape[0], f"Mismatch: X={X_padded.shape}, y={y.shape}"

In [39]:
np.save("../data/abacus_features.npy", X_padded)
np.save("../data/abacus_labels.npy", y)

print(" Saved ABACUS input matrix and labels.")

 Saved ABACUS input matrix and labels.


In [40]:
print("ABACUS matrix and labels saved with matching shapes:", X_padded.shape, y.shape)

ABACUS matrix and labels saved with matching shapes: (17878, 250) (17878,)


In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Step 1: Load the full original dataset again
df = pd.read_csv("../data/fake_jobs.csv")  # go back to raw file to re-clean

# Step 2: Clean the 'description' column manually here
df = df.dropna(subset=['description', 'fraudulent'])  # drop rows with missing desc or labels
df['description'] = df['description'].astype(str)
df['description'] = df['description'].apply(lambda x: x.strip().lower())
df = df[df['description'] != ""]  # remove blanks

# Step 3: Reset index (VERY important for alignment)
df = df.reset_index(drop=True)

# Step 4: Vectorize the cleaned descriptions
texts = df['description'].tolist()
labels = df['fraudulent'].tolist()

vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(texts).toarray()
X_padded = pad_sequences(X, maxlen=250, padding='post', truncating='post')
y = np.array(labels)

# Step 5: Check and save only if they match
assert X_padded.shape[0] == y.shape[0], f"X rows: {X_padded.shape[0]}, y rows: {y.shape[0]}"

# Step 6: Save cleaned versions
np.save("../data/abacus_features.npy", X_padded)
np.save("../data/abacus_labels.npy", y)

print(" Cleaned, aligned, and saved. Shape:", X_padded.shape, y.shape)


 Cleaned, aligned, and saved. Shape: (17878, 250) (17878,)
